In [1]:
# Distance matrixes

In [2]:
pip install haversine

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install vrplib

     -------------------------------------- 204.1/204.1 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import haversine

In [22]:
import pandas as pd
import json
import requests
import numpy as np
import datetime as dt

%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
from src.distance import get_haversine_distance, get_distance_aws

from src.co2 import co2_truck, co2_train
from src.data_matrix import create_data_model, create_dict_points
from src.cvrp_heuristic import find_nearest_customer, construct_initial_solution, local_search, evaluate_cvrp
from src.alns_heuristic import plot_solution

## Functions

In [5]:
def calculate_distance(point1, point2, data):
    # Calculate the distance between two points considering the current load
    # Modify this function to include your own cost calculation logic based on load
    # For example, you can define a function that calculates a cost function based on load-dependent factors
    distance = data['distance_matrix']
    # Modify the distance based on the current load
    # For example, you can add a load-dependent factor to the distance calculation
    modified_distance = distance * (1 + current_load)  # Example modification, adjust as per your requirements
    return modified_distance

def calculate_total_distance(route, data):
    # Calculate the total distance of a route
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += data["distance_matrix"][data["customers"].index(route[i])][data["customers"].index(route[i+1])]
    return total_distance

def total_distance(solution):
    total_distance = 0
    for route in solution:
        route_distance = 0
        for i in range(len(route) - 1):
            distance_arc = data["distance_matrix"][data["customers"].index(route[i])][data["customers"].index(route[i+1])]
            route_distance += distance_arc
        total_distance += route_distance
    return total_distance

In [23]:
class CvrpState:
    """
    Solution state for CVRP. It has two data members, routes and unassigned.
    Routes is a list of list of integers, where each inner list corresponds to
    a single route denoting the sequence of customers to be visited. A route
    does not contain the start and end depot. Unassigned is a list of integers, 
    each integer representing an unassigned customer.
    """

    def __init__(self, routes, unassigned=None):
        self.routes = routes
        self.unassigned = unassigned if unassigned is not None else []

    def copy(self):
        return CvrpState(copy.deepcopy(self.routes), self.unassigned.copy())

    def objective(self):
        """
        Computes the total route costs.
        """
        return sum(route_cost(route) for route in self.routes)
    
    @property
    def cost(self):
        """
        Alias for objective method. Used for plotting.
        """
        return self.objective()

    def find_route(self, customer):
        """
        Return the route that contains the passed-in customer.
        """
        for route in self.routes:
            if customer in route:
                return route

        raise ValueError(f"Solution does not contain customer {customer}.")

## Data Import

In [6]:
df = pd.read_csv('../data/processed/poc3.csv', parse_dates = ["Pickup date"])

df_clients = df[['Receiver longitude', 'Receiver latitude']].drop_duplicates().reset_index()
df_clients.index += 1
df_clients= df_clients.reset_index().drop(["index"], axis = 1).rename({"level_0":"Receiver name"}, axis = 1)
df_clients["Receiver name"] = "C"+df_clients["Receiver name"].astype(str)

df_dc = df[['Shipper longitude', 'Shipper latitude', "DC country"]].drop_duplicates().reset_index()
df_dc.index += 1
df_dc = df_dc.reset_index().drop(["index"], axis = 1).rename({"level_0":"Shipper name"}, axis = 1)
df_dc["Shipper name"] = "DC"+df_dc["Shipper name"].astype(str)

df = df[['Pickup date','Country', 'Sender weight (kg)','Shipper longitude','Shipper latitude', 'Receiver longitude', 'Receiver latitude']].merge(df_clients, on = ['Receiver longitude', 'Receiver latitude'], how = "left").merge(df_dc, on = ['Shipper longitude', 'Shipper latitude'], how = "left")

C:\Users\marle\AppData\Local\Temp\ipykernel_15920\3938569011.py:1: DtypeWarning: Columns (1,5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/processed/poc3.csv', parse_dates = ["Pickup date"])


## Eröffnungsverfahren

In [7]:
date = "2022-01-03"
dc = "DC2"
df_test = df[(df["Shipper name"]==dc)&(df["Pickup date"]>pd.to_datetime("2022-01-02"))&(df["Pickup date"]<pd.to_datetime("2022-01-04"))]
df_test = df_test[df_test["Sender weight (kg)"]>30]

In [8]:
len(df_test["Receiver name"].unique())

92

In [13]:
dict_points = create_dict_points(df_test, "Shipper name", "Shipper latitude", "Shipper longitude")       
dict_points.update(create_dict_points(df_test, "Receiver name", "Receiver latitude", "Receiver longitude")  )

In [14]:
data = create_data_model(df_test, dict_points, 10)

In [15]:
solution = construct_initial_solution(data, 17000)

In [17]:
total_cost = evaluate_cvrp(data, solution)

In [18]:
total_cost

7574.5635130502815

In [24]:
CvrpState(solution)

In [28]:
data["distance_matrix"]

array([[     0, 853612, 623696, ..., 648565, 648323, 648565],
       [851746,      0, 635855, ..., 646397, 646155, 646397],
       [625082, 635435,      0, ...,  27502,  27259,  27502],
       ...,
       [649110, 646039,  28240, ...,      0,   2674,      0],
       [648617, 645546,  27747, ...,   2714,      0,   2714],
       [649110, 646039,  28240, ...,      0,   2674,      0]])

In [98]:
show_map(solution)

In [43]:
[{k: v for v, k in enumerate(data["customers"])}.get(a) for a in solution[2]]

IndexError: list index out of range

# Verbesserungsheuristiken

In [ ]:
local_search(solution)

In [ ]:
#Heuristik laufen lassen

In [ ]:
alns = ALNS(rnd.RandomState(SEED))
alns.add_destroy_operator(random_removal)
alns.add_repair_operator(greedy_repair)

In [99]:
init = nearest_neighbor()
select = RouletteWheel([25, 5, 1, 0], 0.8, 1, 1)
accept = RecordToRecordTravel.autofit(init.objective(), 0.02, 0, 9000)
stop = MaxRuntime(60)

result = alns.iterate(init, select, accept, stop)

NameError: name 'nearest_neighbor' is not defined

In [ ]:
solution = result.best_state
objective = solution.objective()
pct_diff = 100 * (objective - bks.cost) / bks.cost

print(f"Best heuristic objective is {objective}.")
print(f"This is {pct_diff:.1f}% worse than the optimal solution, which is {bks.cost}.")